In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from pathlib import Path
from scipy import special

%matplotlib inline

In [2]:
from cec2017.functions import *

In [3]:
def pop_initiation(pop_size, num_variables, min_val, max_val):
    return np.random.uniform(min_val, max_val, (pop_size, num_variables))

def boundry_checker(pop, min_val, max_val):
    return np.clip(pop, min_val, max_val)

def sorting_pop(pop, obj_func):
    of = obj_func(pop)
    index = np.argmin(of)
    return pop[index], of[index]

def levy_flight(pop, beta, step_size):
    sigma_v = 1
    sigma_u = ((special.gamma(beta+1)*np.sin(np.pi*beta*.5))/(special.gamma((1+beta)*.5)*beta*2**((beta-1)*.5)))**(1/beta)
    v = np.random.normal(0,sigma_v,size=pop.shape)
    u = np.random.normal(0,sigma_u,size=pop.shape)
    s = u/(np.abs(v)**(1/beta))
    return pop + step_size*s

def update_pop(pop, obj_func, beta, step_size, min_val, max_val, R):
    of = obj_func(pop)
    of = np.nan_to_num(of, nan=np.inf)
    pop_indeces = np.argsort(of)
    sorted_pop = pop[pop_indeces]
    result = obj_func([sorted_pop[0]])[0]
    new_pop = sorted_pop.copy()
    new_pop = levy_flight(new_pop, beta, step_size)
    new_pop = boundry_checker(new_pop, min_val, max_val)
    new_pop[:R] = np.where((obj_func(new_pop[:R])<of[:R]).reshape(R,1), new_pop[:R], sorted_pop[:R])
    return new_pop, result

def update_pop_worst(pop, obj_func, beta, step_size, min_val, max_val, R):
    of = obj_func(pop)
    of = np.nan_to_num(of, nan=np.inf)
    pop_indeces = np.argsort(of)
    sorted_pop = pop[pop_indeces]
    result = obj_func([sorted_pop[0]])[0]
    new_pop = sorted_pop.copy()
    new_pop = levy_flight(new_pop, beta, step_size)
    new_pop = boundry_checker(new_pop, min_val, max_val)
    new_pop[-R:] = np.where((obj_func(new_pop[-R:])<of[-R:]).reshape(R,1), new_pop[-R:], sorted_pop[-R:])
    return new_pop, result, sorted_pop[0]

def alg_best(pop_size, iteration, num_variables, min_val, max_val,  obj_func, beta, step_size, R):
    NFE = np.zeros(iteration)
    results = np.zeros(iteration)
    NFE_value = 0
    pop = pop_initiation(pop_size, num_variables, min_val, max_val)
    for i in range(iteration):
        pop, result = update_pop(pop, obj_func, beta, step_size, min_val, max_val, R)
        NFE_value += pop_size
        NFE_value += R
        NFE[i] = NFE_value
        results[i] = result
    X_best, OF_best = sorting_pop(pop, obj_func)
    return X_best, OF_best, results, NFE

def alg_worst(pop_size, iteration, num_variables, min_val, max_val,  obj_func, beta, step_size, R):
    NFE = np.zeros(iteration)
    results = np.zeros(iteration)
    NFE_value = 0
    pop = pop_initiation(pop_size, num_variables, min_val, max_val)
    X_best, OF_best = sorting_pop(pop, obj_func)
    for i in range(iteration):
        pop, temp_result, X_temp = update_pop_worst(pop, obj_func, beta, step_size, min_val, max_val, R)
        NFE_value += pop_size
        NFE_value += R
        NFE[i] = NFE_value
        X_best, OF_best = np.where(OF_best<temp_result, (X_best, OF_best), (X_temp, temp_result))
        results[i] = OF_best
    return X_best, OF_best, results, NFE

<div class='alert alert-info'>
    <b>Psudo Algrotihm Baseline</b>: <i>Pure Random</i>
</div>

In [4]:
def init_genrator(min_val, max_val, num_variables, pop_size):
    return np.random.uniform(min_val, max_val, (pop_size, num_variables))

def boundery_checker(pop, min_val, max_val):
    adj_pop = np.where(pop<min_val, min_val, pop)
    adj_pop = np.where(adj_pop>max_val, max_val, adj_pop)
    return adj_pop

def sorting_pop(pop, obj_func):
    results = obj_func(pop)
    indeces = np.argsort(results)
    return pop[indeces]

def collective_intelligance(pop, vect_lenght):
    rand = np.random.uniform(0, vect_lenght, size=(len(pop), 1))
    elite = pop[0]
    diff = elite - pop
    return pop + rand*diff
   
def pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val):
    NFE_value = 0
    result = np.zeros(iteration)
    NFE = np.zeros(iteration)
    pop = init_genrator(min_val, max_val, num_variables, pop_size)
    for i in range(iteration):
        pop_new = init_genrator(min_val, max_val, num_variables, pop_size)
        overall = np.concatenate((pop, pop_new), axis = 0)
        overall = sorting_pop(overall, obj_func)
        pop = overall[:pop_size]
        NFE_value += (pop_size*2)
        result[i] = obj_func([pop[0]])
        NFE[i] = NFE_value
    return pop[0], obj_func([pop[0]]), result, NFE

##### NC

In [10]:
from cec2017.functions import *

In [11]:
alg_name = 'Pure_Random'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [12]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/NC/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 235.52
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 439.06
---------------------------------------- 

Function:			f3
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 292.83
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s

61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 2229.17
---------------------------------------- 

Function:			f24
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 1569.40
---------------------------------------- 

Function:			f25
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 1641.17
---------------------------------------- 

Function:			f26
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 2713.56
----------------------------------

##### C

In [5]:
def f1(x):
    x = np.array(x)
    x2 = x * x
    return np.sum(x2, axis=1)

def f2(x):
    x = np.array(x)
    ab = np.abs(x)
    a = np.sum(ab, axis=1)
    b = np.prod(ab, axis=1)
    return a + b

def f4(x):
    x = np.array(x)
    ab = np.abs(x)
    return np.max(ab, axis=1)

def f6(x):
    x = np.array(x)
    xnew = x + .5
    floor = np.floor(xnew)
    x2 = floor * floor
    return np.sum(x2, axis=1)

def f7(x):
    x = np.array(x)
    index = np.arange(1,len(x[0])+1)
    ones = np.ones_like(x)
    aa = ones * index
    x2 = x * x
    x4 = x2 * x2
    b = x4 * aa
    return np.sum(b, axis=1)

In [6]:
alg_name = 'Pure_Random'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
num_runs = 100
funcs_index = np.array([1,2,4,6,7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [7]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = pure_random(pop_size, iteration, num_variables, obj_func, min_val, max_val)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 98.35
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 110.04
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 125.17
---------------------------------------- 

Function:			f6
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s]

<div class='alert alert-info'>
    <b>Psudo Algrotihm Type I</b>: <i>Preserve the best</i>
</div>

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take One: Elitism</li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = 1

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                        max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Preserve_take_1'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = 1
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take Two: Collective Elitism for <strong>10%</strong></li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = int(round(.1*pop_size, 0))

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                        max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Preserve_take_2'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.1*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take Three: Collective Elitism for <strong>50%</strong></li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = int(round(.5*pop_size, 0))

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                        max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Preserve_take_3'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.5*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take Four: Collective Elitism for <strong>100%</strong> [<i>greedy strategy</i>]</li>
</ul>
</div>        

<div class='alert alert-danger'>
    Similar for both cases of preserve and eliminate.
</div>

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = pop_size

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                        max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Preserve_take_4'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = pop_size
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-info'>
    <b>Psudo Algrotihm Type II</b>: <i>Eliminate the worst</i>
</div>

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take One: Eliminate</li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = 1

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                         max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Eliminate_take_1'
pop_size = 500
iteration = 10000
R = 1
step_size = 1
beta = 1.5
min_val, max_val = -100, 100
num_variables = 10
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-success'>
    <h4>Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take Two: Collective Elimination for <strong>10%</strong></li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = int(round(.1*pop_size, 0))

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                         max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Eliminate_take_2'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.1*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

<div class='alert alert-danger'>
    <h4>Not Complete</h4>
</div>

<div class='alert alert-warning'>
<ul style="list-style-type:circle">
    <li>Take Three: Collective Elimination for <strong>50%</strong></li>
</ul>
</div>        

In [ ]:
pop_size = 500
iteration = 10000

step_size = 1
beta = 1.5
R = int(round(.5*pop_size, 0))

In [ ]:
func_name = 'f3'
obj_func = eval(func_name)

min_val, max_val = -100, 100
num_variables = 10

In [ ]:
start_time = time.time()

X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                         max_val,  obj_func, beta, step_size, R)

end_time = time.time()
minute = (end_time-start_time)//60
second = (end_time-start_time)-60*minute
print('Best OF', '='*20)
print(OF_best)
print('Variables', '='*18)
print(X_best)
print('Time', '='*22)
print('Time: %d min and %.2f s'%(minute, second))

In [ ]:
fig = plt.plot(NFE, result)
fig = plt.grid(alpha=.5)
fig = plt.margins(0,.05)
fig = plt.axhline(OF_best,ls='--',alpha=.5,c='r')

plt.tight_layout()

<div class='alert alert-success'>
    Independent Runs
</div>

In [ ]:
alg_name = 'Eliminate_take_3'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.5*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.arange(1,31)
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [ ]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

#### C

In [3]:
def f1(x):
    x = np.array(x)
    x2 = x * x
    return np.sum(x2, axis=1)

def f2(x):
    x = np.array(x)
    ab = np.abs(x)
    a = np.sum(ab, axis=1)
    b = np.prod(ab, axis=1)
    return a + b

def f4(x):
    x = np.array(x)
    ab = np.abs(x)
    return np.max(ab, axis=1)

def f6(x):
    x = np.array(x)
    xnew = x + .5
    floor = np.floor(xnew)
    x2 = floor * floor
    return np.sum(x2, axis=1)

def f7(x):
    x = np.array(x)
    index = np.arange(1,len(x[0])+1)
    ones = np.ones_like(x)
    aa = ones * index
    x2 = x * x
    x4 = x2 * x2
    b = x4 * aa
    return np.sum(b, axis=1)

In [4]:
alg_name = 'Preserve_take_1'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = 1
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [5]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 488.55
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 496.60
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 474.75
---------------------------------------- 

Function:			f6
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s

In [6]:
alg_name = 'Preserve_take_2'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.1*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [7]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 478.58
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 497.84
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 510.83
---------------------------------------- 

Function:			f6
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s

In [8]:
alg_name = 'Preserve_take_3'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.5*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [9]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 465.69
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 487.36
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 488.27
---------------------------------------- 

Function:			f6
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s

In [10]:
alg_name = 'Preserve_take_4'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = pop_size
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [11]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_best(pop_size, iteration, num_variables, min_val,
                                                max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 485.94
---------------------------------------- 

Function:			f2
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 504.80
---------------------------------------- 

Function:			f4
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s] done.
----------------------------------------
Finished in: 500.58
---------------------------------------- 

Function:			f6
1 run[s] done.
11 run[s] done.
21 run[s] done.
31 run[s] done.
41 run[s] done.
51 run[s] done.
61 run[s] done.
71 run[s] done.
81 run[s] done.
91 run[s

In [3]:
alg_name = 'Eliminate_take_1'
pop_size = 500
iteration = 10000
R = 1
step_size = 1
beta = 1.5
min_val, max_val = -100, 100
num_variables = 10
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [4]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

11 run[s] done.


...

81 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

91 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

----------------------------------------
Finished in: 739.57
---------------------------------------- 



In [5]:
alg_name = 'Eliminate_take_2'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.1*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [6]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

11 run[s] done.


...

81 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

91 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

----------------------------------------
Finished in: 741.68
---------------------------------------- 



In [7]:
alg_name = 'Eliminate_take_3'
pop_size = 500
iteration = 10000
min_val, max_val = -100, 100
num_variables = 10
R = int(round(.5*pop_size, 0))
step_size = 1
beta = 1.5
num_runs = 100
funcs_index = np.array([1, 2, 4, 6, 7])
root_directory = 'C:/Users/bz267/Optimization results/Eliminate vs Preserve'

In [8]:
for j in funcs_index:
    func_name = 'f%d'%j
    obj_func = eval(func_name)
    print('=='*20)
    print('Function:\t\t\t%s'%func_name)
    print('=='*20)
    func_s_time = time.time()
    overall_result = pd.DataFrame(columns=['best_OF', 'NFE', 'time (s)', 'Best DVs'])
    for m in range(num_runs):
        start_time = time.time()
        
        X_best, OF_best, result, NFE = alg_worst(pop_size, iteration, num_variables, min_val,
                                                 max_val,  obj_func, beta, step_size, R)
        
        end_time = time.time()
        temp_db = pd.DataFrame(pd.Series([OF_best, NFE[-1], end_time-start_time, X_best])).transpose()
        temp_db.columns = ['best_OF', 'NFE', 'time (s)', 'Best DVs']
        overall_result = pd.concat((overall_result, temp_db), axis=0, ignore_index=True)
        if m%10==0:
            print('%d run[s] done.'%(m+1))
    func_e_time = time.time()
    file_name = '%s.csv'%func_name
    folder_name = '/%s/C/'%alg_name
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    overall_result.to_csv(output_dir / file_name)
    print('--'*20)
    print('Finished in: %.2f'%(func_e_time-func_s_time))
    print('--'*20,'\n')

Function:			f1


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

11 run[s] done.


...

81 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

91 run[s] done.


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested 

----------------------------------------
Finished in: 839.81
---------------------------------------- 

